In [28]:
%autosave 45
#!/usr/bin/python
# -*- coding: utf-8 -*-

import tweepy
import spacy
import csv
import json
import pandas as pd
import os

Autosaving every 45 seconds


In [4]:
# Twitter API credentials
credentialsPath = r'..\0_data\credentials'
with open(os.path.join(credentialsPath, 'twitter_credentials.json')) as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

# Create the api endpoint
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [5]:
# Convert to data frame and capture each column feature in an array
def toDataFrame(tweets):
    DataSet = pd.DataFrame()
    
    tweetsText = []
    for tweet in tweets:
            if 'retweeted_status' in  dir(tweet):
                tweetsText.append(tweet.retweeted_status.full_text.encode('utf-8'))                
            else:
                tweetsText.append(tweet.full_text.encode('utf-8'))
            
    DataSet['Text'] = [text for text in tweetsText]
    DataSet['User'] = [tweet.user.name.encode('utf-8') for tweet in tweets]
    
    tweetsImages = []
    for tweet in tweets:
        if 'media' in tweet.entities:
            for image in tweet.entities['media']:
                tweetsImages.append(image['media_url'])
        else:
            tweetsImages.append('')
            
    
    DataSet['Image Urls'] = [image for image in tweetsImages]
            
    tweetsLongitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLongitudes.append(tweet.coordinates["coordinates"][0])
        else:
            tweetsLongitudes.append('')
    DataSet['Longitude'] = [longitude for longitude in tweetsLongitudes]
    
    tweetsLatitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLatitudes.append(tweet.coordinates["coordinates"][1])
        else:
            tweetsLatitudes.append('')
    DataSet['Latitude'] = [latitude for latitude in tweetsLatitudes]
    
    return DataSet


# Specify the maximum number of tweets that you want to be extracted.
maximum_number_of_tweets_to_be_extracted = \
    int(input('Enter the number of tweets that you want to extract- '))

# Specify if you are looking for hashtags or mentions
# Note: mentions will get you exactly a mention and are case insensitive
# Note 2: hashtags are case insensitive and not required
# Example: datascience, Datascience, #datascience, and #Datascience will return tweets about data science
# Hashtags of course are explicit mentions of a term, but are not case sensitive on Twitter
tag_choice = input('Enter which you are searching for, a hashtag (#), or a mention (@)- ')

# Specify the term you want to scrape
mention = input('Enter the term you want to scrape- ')

results = []

for tweet_info in tweepy.Cursor(api.search, q=str(tag_choice) + mention,
                           tweet_mode='extended').items(maximum_number_of_tweets_to_be_extracted):
    results.append(tweet_info)
    
for tweet_info in tweepy.Cursor(api.search, q=str(tag_choice) + mention.upper(),
                           tweet_mode='extended').items(maximum_number_of_tweets_to_be_extracted):
    results.append(tweet_info)

data = toDataFrame(results)
outputPath = r'..\0_data\manual'
filePath = os.path.join(outputPath,'tweets_with_mention_' + mention + '.csv')
if not os.path.isfile(filePath):
    data.to_csv(filePath, index=False)
else:
    with open(filePath, 'a') as file:
        data.to_csv(file, index = False)
print ('Extracted ' + str(maximum_number_of_tweets_to_be_extracted) \
    + ' tweets with ' + str(tag_choice) + mention)

Enter the number of tweets that you want to extract- 40
Enter which you are searching for, a hashtag (#), or a mention (@)- #
Enter the term you want to scrape- lol
Extracted 40 tweets with #lol


In [6]:
# Load the spacy library and English corpus
nlp = spacy.load('en')

nlp.pipeline

[('tagger', <spacy.pipeline.Tagger at 0x119cccf0>),
 ('parser', <spacy.pipeline.DependencyParser at 0xfa04ed0>),
 ('ner', <spacy.pipeline.EntityRecognizer at 0xfa91ed0>)]

In [25]:
# Instantiate the clean text and user arrays
clean_texts = []
clean_users = []

# Clean the text data
for row in data.itertuples():
    text = row[1].decode('utf-8')
    user = row[2].decode('utf-8')
    
    text_doc = nlp.make_doc(text)
    tokens = [token for token in text_doc if not token.is_stop]
    tokens = [token for token in tokens if not token.is_punct]
    tokens = [token for token in tokens if token.text != ' ']    
    tokenz_final = [token.text for token in tokens]
    
    text = " ".join(tokenz_final).strip()
    text = text.replace('\n', '')
    
    clean_texts.append(text)
    clean_users.append(user)

In [26]:
clean_data.head()

,clean_texts,clean_users
0,Repost epicfunnypage00 make_repost Omg hahah...,Epic Fail
1,@jpuopolo JacobWohl Hilarious 15minutesAreUp...,H2O4US
2,@ThePoniExpress Love Andy Dumbest poll fuckthi...,Torso Macabre
3,Two kids coat funny lol https://t.co/ycFRxUSWd1,Sasha
4,When iPhone ’s Battery Bar Goes Red funny Funn...,anuri
